In [1]:
import argparse
from pathlib import Path

simple_hyp_params = argparse.Namespace(batch_size= 2,d_model = 32,
early_stopping_activated = False,early_stop_steps_per_half_clr_cycle = 3,
nb_epochs = 300)

dset_truncation =argparse.Namespace(limit_length= 10,use_splitting = True,
max_length_from_file = False)


opt_params = argparse.Namespace(base_lr = 10**(-6),max_lr = 0.0005,
momentum = 0.9,half_period_cycle = 5,
)

train_state_control = argparse.Namespace(load_from_backup = False,
restore_from_backup = tuple(["model_params","scheduler","optimizer","losses","metrics"])
)


path_to_root = Path("../../")

paths = argparse.Namespace(path_dataset = "data/french_english_dataset/fra.txt",
path_language_info = "models/language_info.pth",
path_dataset_splitting = "dataset_splitting",
path_model_and_dependencies = "models/sequence_translator_transformer.pth"
)

for key,path in paths.__dict__.items():
    paths.__dict__[key] = str(path_to_root.joinpath(path))



In [2]:
from ploomber_engine.ipython import PloomberClient
from ploomber.tasks import NotebookRunner
from ploomber import DAG
from pathlib import Path
from ploomber.products import File

# initialize client
client = PloomberClient.from_path(Path("./training_setup.ipynb"))
from argparse import Namespace

from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

initial_namespace = argparse.Namespace(**{key:globals()[key] for key in ["simple_hyp_params","dset_truncation",
                                                  "opt_params","train_state_control",
                                                  "paths"]})

train_setup = client.get_namespace(initial_namespace.__dict__)
for key,val in train_setup.items():
    globals()[key] = val

Executing cell: 12: 100%|███████████████████████| 19/19 [00:02<00:00,  7.34it/s]


In [3]:
import matplotlib.pyplot as plt

best_loss_val_mean = np.inf
best_epoch = scheduler.last_epoch

for epoch in tqdm(range(simple_hyp_params.nb_epochs)):
    #import time
    #start = time.time()
    print(f"optimizing for epoch {epoch}")
    print("training_step")
    loss_train,nb_words_per_batch_train,metric_train = model_trainer.train_on_epoch()
    
    print("validation_step")
    loss_val,nb_words_per_batch_val,metric_val = model_trainer.validate_on_epoch()

    loss_train = np.array([float(el) for el in loss_train])
    loss_val = np.array([float(el) for el in loss_val])
    train_weights = 1/sum(nb_words_per_batch_train)
    val_weights = 1/sum(nb_words_per_batch_val)
    
    
        
    losses["train"].append(np.sum(loss_train)/sum(nb_words_per_batch_train))
    losses["val"].append(np.sum(loss_val)/sum(nb_words_per_batch_val))
    metrics["train"].append(metric_train)
    metrics["val"].append(metric_val)
    
    current_loss_val_mean = np.mean(loss_val)
    if (current_loss_val_mean < best_loss_val_mean):
        best_epoch = scheduler.last_epoch
        best_loss_val_mean = current_loss_val_mean

        model_training_state = {"model_params":model_trainer.model.state_dict(),
                               "model_inputs":model_inputs,
                              "optimizer":optimizer.state_dict(),
                              "scheduler":scheduler.state_dict(),
                              "losses":losses,
                              "metrics":metrics
                              }
        
        torch.save(model_training_state,path_model_and_dependencies)
        print(f"saving for epoch {epoch}")

        plt.plot(losses["train"],"b*")
        plt.plot(losses["val"],"g*")
        plt.title("losses")
        plt.show()
        break
        #import pdb;pdb.set_trace()
    elif epoch - best_epoch > early_stop_thresh  and early_stopping_activated:
        print("Early stopped training at epoch %d" % epoch)
        break  # terminate the training loop
    #stop = time.time()
    #print(stop-start)

    del loss_train,nb_words_per_batch_train,metric_train

    del loss_val,nb_words_per_batch_val,metric_val


  0%|                                                                                             | 0/300 [00:00<?, ?it/s]

optimizing for epoch 0
training_step
0 7.143755594889323
validation_step
0 7.198941230773926
saving for epoch 0
Figure(640x480)


  0%|                                                                                             | 0/300 [00:02<?, ?it/s]


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(losses["train"],"b*")
plt.plot(losses["val"],"g*")
plt.title("losses")
plt.show() 

Figure(640x480)
